# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 59 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.14455


extracting tarball to tmp_2208.14455...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.14480


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.14455/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'authors' from 'tmp_2208.14455/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'affiliation' from 'tmp_2208.14455/affiliation.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2208.14927...

 done.


Retrieving document from  https://arxiv.org/e-print/2208.14988


extracting tarball to tmp_2208.14988...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14927-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14927) | **Experimental determination of the dissociative recombination rate  coefficient for rotationally-cold CH$^{+}$ and its implications for the  diffuse cloud chemistry**  |
|| Daniel Paul, et al. -- incl., <mark>Holger Kreckel</mark> |
|*Appeared on*| *2022-09-01*|
|*Comments*| *Main paper: PDFLaTeX with 7 pages, 3 figures. Appendix starting on page 7: PDFLaTeX with 11 pages, 2 figures, 4 tables. This article has been accepted by The Astrophysical Journal*|
|**Abstract**| Observations of CH$^+$ are used to trace the physical properties of diffuse clouds, but this requires an accurate understanding of the underlying CH$^+$ chemistry. Until this work, the most uncertain reaction in that chemistry was dissociative recombination (DR) of CH$^+$. Using an electron-ion merged-beams experiment at the Cryogenic Storage Ring, we have determined the DR rate coefficient of the CH$^+$ electronic, vibrational, and rotational ground state applicable for different diffuse cloud conditions. Our results reduce the previously unrecognized order-of-magnitude uncertainty in the CH$^+$ DR rate coefficient to $\sim \pm 20\%$ and are applicable at all temperatures relevant to diffuse clouds, ranging from quiescent gas to gas locally heated by processes such as shocks and turbulence. Based on a simple chemical network, we find that DR can be an important destruction mechanism at temperatures relevant to quiescent gas. As the temperature increases locally, DR can continue to be important up to temperatures of $ \sim 600\,\mathrm{K} $ if there is also a corresponding increase in the electron fraction of the gas. Our new CH$^+$ DR rate coefficient data will increase the reliability of future studies of diffuse cloud physical properties via CH$^+$ abundance observations. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14519-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14519) | **Gemini North Adaptive Optics (GNAO) facility overview and status updates**  |
|| Gaetano Sivo, et al. -- incl., <mark>Célia Blain</mark>, <mark>Henry Roe</mark>, <mark>Heather Carr</mark>, <mark>Anja Feldmeier-Krause</mark> |
|*Appeared on*| *2022-09-01*|
|*Comments*| *SPIE conference 2022 Montreal*|
|**Abstract**| The Gemini North Adaptive Optics (GNAO) facility is the upcoming AO facility for Gemini North providing a state-of-the-art AO system for surveys and time domain science in the era of JWST and Rubin operations. GNAO will be optimized to feed the Gemini infrared Multi Object Spectrograph (GIRMOS). While GIRMOS is the primary science driver for defining the capabilities of GNAO, any instrument operating with an f/32 beam can be deployed using GNAO. The GNAO project includes the development of a new laser guide star facility which will consist of four side-launched laser beams supporting the two primary AO modes of GNAO: a wide-field mode providing an improved image quality over natural seeing for a 2-arcminute circular field-of-view and a narrow-field mode providing near diffraction-limited performance over a 20x20 arcsecond square field-of-view. The GNAO wide field mode will enable GIRMOS's multi-IFU configuration in which the science beam to each individual IFU will be additionally corrected using multi-object AO within GIRMOS. The GNAO narrow field mode will feed the GIRMOS tiled IFU configuration in which all IFUs are combined into a "super"-IFU in the center of the field. GNAO also includes the development of a new Real Time Controller, a new GNAO Facility System Controller and finally the development of a new AO Bench. We present in this paper an overview of the GNAO facility and provide a status update of each product. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14632-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14632) | **A Wide and Deep Exploration of Radio Galaxies with Subaru HSC (WERGS).  IX. The Most Overdense Region at z~5 Inhabited by a Massive Radio Galaxy**  |
|| Hisakazu Uchiyama, et al. -- incl., <mark>Chien-Hsiu Lee</mark> |
|*Appeared on*| *2022-09-01*|
|*Comments*| *7 pages, 3 figures, accepted in PASJ*|
|**Abstract**| We report on the galaxy density environment around a high-z radio galaxy (HzRG) at z=4.72, HSC J083913.17+011308.1 (HSC J0839+0113), probed using an r-dropout Lyman break galaxy (LBG) sample from the Hyper Suprime-Cam Subaru Strategic Program data. We find that HSC J0839+0113 resides in the outskirt of an overdense region identified by the r-dropout galaxies at a 4.7 sigma significance level. The projected distance between HSC J0839+0113 and the peak position of the overdense region is 0.4 physical Mpc which is shorter than the typical protocluster radius in this epoch. According to the extended Press Schechter and the light cone models, the HSC J0839+0113-hosted overdense region is expected to evolve into a halo > 10^14 Msun at z=0 with a high probability of >80 %. These findings suggest that HSC J0839+0113 is associated with a protocluster. The HSC J0839+0113 rich-system is the most overdense region of LBGs among the known protoclusters with LBGs in the same cosmic epoch. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14904-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14904) | **The on-ground data reduction and calibration pipeline for SO/PHI-HRT**  |
|| J. Sinjan, et al. -- incl., <mark>A. Alvarez-Herrero</mark>, <mark>K. Heerlein</mark>, <mark>T. Lange</mark>, <mark>R. Müller</mark> |
|*Appeared on*| *2022-09-01*|
|*Comments*| **|
|**Abstract**| The ESA/NASA Solar Orbiter space mission has been successfully launched in February 2020. Onboard is the Polarimetric and Helioseismic Imager (SO/PHI), which has two telescopes, a High Resolution Telescope (HRT) and the Full Disc Telescope (FDT). The instrument is designed to infer the photospheric magnetic field and line-of-sight velocity through differential imaging of the polarised light emitted by the Sun. It calculates the full Stokes vector at 6 wavelength positions at the Fe I 617.3 nm absorption line. Due to telemetry constraints, the instrument nominally processes these Stokes profiles onboard, however when telemetry is available, the raw images are downlinked and reduced on ground. Here the architecture of the on-ground pipeline for HRT is presented, which also offers additional corrections not currently available on board the instrument. The pipeline can reduce raw images to the full Stokes vector with a polarimetric sensitivity of $10^{-3}\cdot I_{c}$ or better. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14914-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14914) | **Object classification on video data of meteors and meteor-like  phenomena: algorithm and data**  |
|| Rabea Sennlaub, et al. -- incl., <mark>Thomas Müller</mark> |
|*Appeared on*| *2022-09-01*|
|*Comments*| *11 Pages, 10 Figures, Accepted for publication in MNRAS Journal*|
|**Abstract**| Every moment, countless meteoroids enter our atmosphere unseen. The detection and measurement of meteors offer the unique opportunity to gain insights into the composition of our solar systems' celestial bodies. Researchers, therefore, carry out a wide-area-sky-monitoring to secure 360-degree video material, saving every single entry of a meteor. Existing machine intelligence cannot accurately recognize events of meteors intersecting the earth's atmosphere due to a lack of high-quality training data publicly available. This work presents four reusable open source solutions for researchers trained on data we collected due to the lack of available labeled high-quality training data. We refer to the proposed dataset as the NightSkyUCP dataset, consisting of a balanced set of 10,000 meteor- and 10,000 non-meteor-events. Our solutions apply various machine learning techniques, namely classification, feature learning, anomaly detection, and extrapolation. For the classification task, a mean accuracy of 99.1\% is achieved. The code and data are made public at figshare with DOI: 10.6084/m9.figshare.16451625 |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14988-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14988) | **CMB lensing with shear-only reconstruction on the full sky**  |
|| <mark>Frank J. Qu</mark>, Anthony Challinor, Blake D. Sherwin |
|*Appeared on*| *2022-09-01*|
|*Comments*| **|
|**Abstract**| Reconstruction of gravitational lensing effects in the CMB from current and upcoming surveys is still dominated by temperature anisotropies. Extragalactic foregrounds in temperature maps can induce significant biases in the lensing power spectrum obtained with the standard quadratic estimators. Techniques such as masking cannot remove these foregrounds fully, and the residuals can still lead to large biases if unaccounted for. In this paper, we study the "shear-only" estimator, an example of a class of geometric methods that suppress extragalactic foreground contamination while making only minimal assumptions about foreground properties. The shear-only estimator has only been formulated in the flat-sky limit and so is not easily applied to wide surveys. Here, we derive the full-sky version of the shear-only estimator and its generalisation to an $m=2$ multipole estimator that has improved performance for lensing reconstruction on smaller scales. The multipole estimator is generally not separable, and so is expensive to compute. We explore separable approximations based on a singular-value decomposition, which allow efficient evaluation of the estimator with real-space methods. Finally, we apply these estimators to simulations that include extragalactic foregrounds and verify their efficacy in suppressing foreground biases. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14664-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14664) | **LOPES 3D -- studies on the benefits of EAS-radio measurements with  vertically aligned antennas**  |
|| <mark>D. Huber</mark>, et al. -- incl., <mark>D. Heck</mark>, <mark>M. Roth</mark> |
|*Appeared on*| *2022-09-01*|
|*Comments*| *Submission to the proceedings of the ARENA2014 workshop, which unfortunately were never published; thus documented here for reference*|
|**Abstract**| The LOPES experiment was a radio interferometer built at the existing air shower array KASCADE-Grande in Karlsruhe, Germany. The last configuration of LOPES was called LOPES 3D and consisted of ten tripole antennas. Each of these antennas consisted of three crossed dipoles east-west, north-south, and vertically aligned. With this, LOPES 3D had the unique possibility to study the benefits of measurements with vertically aligned antennas in the environment of the well understood and calibrated particle detector array KASCADE-Grande. The measurements with three spatially coincident antennas allows a redundant reconstruction of the electric field vector. Several methods to exploit the redundancy were developed and tested. Furthermore, for the first time in LOPES, the background noise could be studied polarization- and direction dependent. With LOPES 3D it could be demonstrated that radio detection reaches a higher efficiency for inclined showers when including measurements with vertically aligned antennas and that the vertical component gets more important for the measurement of inclined showers. In this contribution we discuss a weighting scheme for the best combination of three redundant reconstructed electric field vectors. Furthermore, we discuss the influence of these weighting schemes on the ability to reconstruct air showers using the radio method. We show an estimate of the radio efficiency for inclined showers with focus on the benefits of measurements with vertically aligned antennas and we present the direction dependent noise in the different polarizations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.14875-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.14875) | **RISTRETTO: Seven Spaxels Single Mode Spectrograph Design**  |
|| Bruno Chazelas, et al. -- incl., <mark>Ian Hughes</mark> |
|*Appeared on*| *2022-09-01*|
|*Comments*| *7 pages, 7 figures, SPIE Astronomical Telescope and Instrumentation 2022, conference : Ground-based and Airborne Instrumentation for Astronomy IX (12184)*|
|**Abstract**| The RISTRETTO project is aiming to build an instrument that will detect the reflected light from close-by exoplanet. It is a two stage instrument: An extreme AO system in the visible, followed by a seven spaxel single mode High resolution Spectrograph. In this paper we present the design of this spectrograph: a classical echelle spectrograph fed with single mode fibers. Standard single mode fibers have been chosen and are forming a long tilted slit in order to have the right order spacing on the detector. The instrument will be under vacuum and thermally controlled in order to make it stable. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.14927.md
    + _build/html/tmp_2208.14927/./P3.png
    + _build/html/tmp_2208.14927/./P1.png
    + _build/html/tmp_2208.14927/./P2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

#  and its implications for the diffuse cloud chemistry

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.14927-b31b1b.svg)](https://arxiv.org/abs/2208.14927)<mark>Appeared on: 2022-09-01</mark> - _Main paper: PDFLaTeX with 7 pages, 3 figures. Appendix starting on page 7: PDFLaTeX with 11 pages, 2 figures, 4 tables. This article has been accepted by The Astrophysical Journal_

</div>
<div id="authors">

Daniel Paul, et al. -- incl., <mark>Holger Kreckel</mark>

</div>
<div id="abstract">

**Abstract:** Observations of CH$^+$are used to trace the physical properties of diffuse clouds, but this requires an accurate understanding of the underlying CH$^+$chemistry. Until this work, the most uncertain reaction in that chemistry was dissociative recombination (DR) of CH$^+$. Using an electron-ion merged-beams experiment at the Cryogenic Storage Ring, we have determined the DR rate coefficient of the CH$^+$electronic, vibrational, and rotational ground state applicable for different diffuse cloud conditions. Our results reduce the previously unrecognized order-of-magnitude uncertainty in the CH$^+$DR rate coefficient to$\sim \pm 20\%$and are applicable at all temperatures relevant to diffuse clouds, ranging from quiescent gas to gas locally heated by processes such as shocks and turbulence. Based on a simple chemical network, we find that DR can be an important destruction mechanism at temperatures relevant to quiescent gas. As the temperature increases locally, DR can continue to be important up to temperatures of$ \sim\SI{600}{K} $if there is also a corresponding increase in the electron fraction of the gas. Our new CH$^+$DR rate coefficient data will increase the reliability of future studies of diffuse cloud physical properties via CH$^+$abundance observations.

</div>

<div id="div_fig1">

<img src="tmp_2208.14927/./P3.png" alt="Fig4" width="100%"/>

**Figure 4. -** CH$^{+}$ DR kinetic temperature rate coefficient and its relevance for the CH$^{+}$ destruction in diffuse interstellar clouds. (a) The thick solid line shows the experimentally determined $ J=0 $ kinetic temperature rate coefficient of this work. The one-sigma error band in gray represents the quadrature sum of all systematic uncertainties.  The dashed lines indicate rate coefficients from the UMIST (\citealt{McElroy2013}, upper line) and KIDA (\citealt{Wakelam2012}, lower line) astrochemistry databases. The calculated $ J=0 $ rate coefficient of \citet{Mezei2019} is drawn as dotted line. (b) Diffuse cloud parameter ranges where DR is the dominant CH$^+$ destruction process (colored) for a standard electron fraction ($ \chi_{\mathrm{e}}=\SI{1.6e-4}  $) and for enhanced values of $ \chi_{\mathrm{e}} $ as given. One-sigma error bars for $ f_\mathrm{H_2}=1 $ and $ f_\mathrm{H_2}=0 $ are shown at the top and bottom of the figure, respectively. They were calculated from the combined systematic uncertainties for the rate coefficients $ \alpha_\mathrm{H} $($\sim50\%$, \cite{Plasil2011}), $ \alpha_\mathrm{H_2} $($\sim20\%$, \cite{Gerlich2011}), and $ \alpha^\mathrm{k}_{J=0} $($ \sim20\% $, this work). (*fig:plasmarate*)

</div>
<div id="div_fig2">

<img src="tmp_2208.14927/./P1.png" alt="Fig2" width="100%"/>

**Figure 2. -** (a) Schematic of the CSR experimental setup for DR measurements of CH$^+$  and probing of its internal level populations. See text for details. (b) Measured relative $ J $-level populations in the CSR for storage times of $ \SI[parse-numbers=false]{60-80}{s} $ with statistical one-sigma error bars. (c) Relative $ J $-level populations predicted in TSR for a thermal equilibrium at $ T=\SI{300}{K} $. (*fig:setup*)

</div>
<div id="div_fig3">

<img src="tmp_2208.14927/./P2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Merged-beams DR rate coefficient (blue line) for the CH$^{+}$ ground state $X^1\Sigma^+ (v=0,J=0)$ with the statistical one-sigma uncertainties (blue shaded area). The one-sigma uncertainty of the absolute scale is $ \pm13 \% $. The room-temperature TSR results of \citet{Amitay1996}, which have a $ \pm50 \% $ absolute scaling systematic uncertainty, are shown by the gray squares. The symbols and lines on top of the figure indicate the full width at half maximum (FWHM) energy resolution $ \Delta E $ of the CSR and TSR measurements. The divergence of the CSR from the TSR data for $ E_\mathrm{d}\gtrsim\SI{10}{eV} $ is due to contribution from dissociative excitation (see text). The data behind the TSR and CSR graphs are both available. The latter are part of Figure \ref{fig:ratecoeffmeas}(b). (*fig:ratecoeff*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

21  publications in the last 7 days.
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers